## Stroboscopic plot for Mean field 

In [ ]:
import numpy as np
from numpy.linalg import multi_dot, norm, eig
from odeintw import odeintw
from scipy.signal import find_peaks
import time
import math

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

fs = 15
pheightmin = 10

sx = np.array([[0, 1],[ 1, 0]])
sy = np.array([[0, -1j],[1j, 0]])
sz = np.array([[1, 0],[0, -1]])

t = np.linspace(0, 1000, 30000)
h0 = 0.1
h = 25.0                          # for number of stroning cycles
besj_zero = 2.40482555769577

#omegas = [3.0, 5.55, 10.396, 25.0]
omegas = [3.0, 5.55]
fig, axs = plt.subplots(len(omegas), 2, sharex='col', figsize=(14,6))
lastplot = len(omegas) - 1

def mf_jac (psi, t, h0, h, omega):
    sx_mf = multi_dot([psi.conjugate(), sx, psi])
    drive = h0 + h * np.cos(omega * t)
    jac = (1j) * (sx_mf * sx + drive * sz)
    return jac

def mf_func (psi, t, h0, h, omega):
    return np.dot(mf_jac(psi, t, h0, h, omega), psi)

def tls_jac (psi, t, h0, h, omega):
    drive = h0 + h * np.cos(omega * t)
    jac = (1j) * 0.5 * (sx + drive * sz)
    return jac

def tls_func (psi, t, h0, h, omega):
    return np.dot(tls_jac(psi, t, h0, h, omega), psi)

#Start from the t=0 ground state
#evals, evecs = eig((1j) * tls_jac(None, 0.0, h0, h, 1.0))
#psi0 = evecs[:,np.argmin(evals)].copy()

#Start from eigenstate of sx
evals, evecs = eig((1j) * tls_jac(None, 0.0, 0.0, 0.0, 0.0))
psi0 = evecs[:,np.argmin(evals)].copy()
start = time.time()

for i, w in enumerate(omegas):
    wr = w/(2 * np.pi)
    T = 2 * np.pi/wr
    print("T",T)
    #t = np.linspace(0, T * cycles, cycles+1)
    tcycl = math.floor(t[len(t)-1]/T)
    print("tcycl=",tcycl)
    tcl = np.arange(1,tcycl+1)
    print("tcl=",tcl)
    sxx,szz = np.zeros(tcycl+1), np.zeros(tcycl+1)
    print("sxx size",len(sxx))
    
    # Set up the inputs and call odeintw to solve the system.
    sol_mf = odeintw(mf_func, psi0, t, args=(h0, h, w), Dfun=mf_jac)
    #sol_tls = odeintw(tls_func, psi0, t, args=(h0, h, w), Dfun=tls_jac)
    #calculate expectation values
    mx_mf = np.einsum("ij,jk,ik->i", sol_mf.conjugate(), sx, sol_mf)
    mz_mf = np.einsum("ij,jk,ik->i", sol_mf.conjugate(), sz, sol_mf)
    #mx_tls = np.einsum("ij,jk,ik->i", sol_tls.conjugate(), sx, sol_tls)
    #mz_tls = np.einsum("ij,jk,ik->i", sol_tls.conjugate(), sz, sol_tls)
    
    for xx in tcl:
        #print("xx=",xx)
        sxx[xx] = mx_mf[math.floor((len(mx_mf)-1)/(xx * T))]
        szz[xx] = mx_mf[math.floor((len(mz_mf)-1)/(xx * T))]
        #print(xx,szz[xx])
    
    axs[i, 0].set_title(r"$\omega_D = {}$, ".format(w) + r"$h_D = {}$ ".format(h), fontsize=fs)
    #axs[i, 0].plot(t, mz_tls.real,'r', label = "TLS Soln")
    #axs[i, 0].plot(tcl, mz_mf.real, 'b', label = "MF Soln", alpha=0.7)
    axs[i, 0].plot(tcl, szz[1:len(szz)].real, 'b', label = "MF Soln", alpha=0.7)
    axs[i, 0].scatter(tcl, szz[1:len(szz)].real, label = "MF Soln", color='red', marker='.')
    #axs[i].scatter(tcl, mz_mf.real, label = "MF Soln", color='red', marker='.')
    axs[i, 0].set_ylabel(r'$\langle\sigma^z\rangle$',fontsize=fs, rotation=0)
    axs[i, 0].tick_params(direction='in', length=8, width=3, labelsize=fs)
    axs[i, 0].set_yticks([-1.,-0.5,0.,0.5,1.], minor=False)
    axs[i, 0].set_xlim(-1.1, 50.0)
    #axs[i, 0].set_ylim(-1.1, 1.1)
    
    axs[i, 1].set_title(r"$\omega_D = {}$, ".format(w) + r"$h_D = {}$ ".format(h), fontsize=fs)
    #axs[i, 0].plot(t, mz_tls.real,'r', label = "TLS Soln")
    #axs[i, 0].plot(tcl, mz_mf.real, 'b', label = "MF Soln", alpha=0.7)
    axs[i, 1].plot(tcl, szz[1:len(szz)].real, 'b', label = "MF Soln", alpha=0.7)
    axs[i, 1].scatter(tcl, szz[1:len(szz)].real, label = "MF Soln", color='red', marker='.')
    #axs[i].scatter(tcl, mz_mf.real, label = "MF Soln", color='red', marker='.')
    axs[i, 1].set_ylabel(r'$\langle\sigma^z\rangle$',fontsize=fs, rotation=0)
    axs[i, 1].tick_params(direction='in', length=8, width=3, labelsize=fs)
    axs[i, 1].set_yticks([-1.,-0.5,0.,0.5,1.], minor=False)
    #axs[i, 1].set_xlim(-1.1, 50.0)
    #axs[i, 1].set_ylim(-1.1, 1.1)
    

axs[1, 0].set_xlabel('numner of strobe-cycles',fontsize=fs)  
axs[1, 1].set_xlabel('numner of strobe-cycles',fontsize=fs)  

legend0 = axs[0, 1].legend(prop={'size': fs})

# Norm sanity check
#plt.plot(t, np.abs(sol_mf[:, 0])**2 + np.abs(sol_mf[:, 1])**2)
fig.tight_layout()
fig.savefig("stroboscopic_plot.jpeg",dpi=200)
print("time taken",(time.time()-start)/60,"minutes")
plt.plot()